# Artificial Neural Networks and Deep Learning

---

## ⚙️ Import Libraries

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

%pip install keras-cv
import keras_cv
import keras

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Import other libraries
from tensorflow import data as tf_data
import cv2
import pandas as pd
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from PIL import Image
import matplotlib.gridspec as gridspec
import requests
from io import BytesIO

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

Note: you may need to restart the kernel to use updated packages.


## ⏳ Load the Data

In [ ]:
data = np.load('/kaggle/input/balanced-augmented-dataset0-1/balanced_augmented_dataset(0_1).npz')
X_train = data['images']
y_train = data['labels']

label_names = {
0: 'Basophil',
1: 'Eosinophil',
2: 'Erythroblast',
3: 'Immature granulocytes',
4: 'Lymphocyte',
5: 'Monocyte',
6: 'Neutrophil',
7: 'Platele'
}

## 🔄 Process Data

In [ ]:
# print the shapes of the resulting datasets
print("Training Data Shape:", X_train.shape)
print("Training Label Shape:", y_train.shape)

# de-normalize dataset
X_train *= 255

Training Data Shape: (51943, 96, 96, 3)
Training Label Shape: (51943, 8)


## 🧮 Define Network Parameters

In [ ]:
# Input shape for the model
input_shape = X_train.shape[1:]

# Output shape for the model
output_shape = y_train.shape[1]

In [ ]:
# Batch size for training
batch_size = 128

# Learning rate: step size for updating the model's weights
learning_rate = 1e-4

# Dropout rate
dropout_rate = 0.5

## 🛠️ Build the Model

In [ ]:
# Automatically sets all layers to use float16 computations wherever possible
tfk.mixed_precision.set_global_policy('mixed_float16')

convbase = tfk.applications.ConvNeXtBase(
    input_shape=(96, 96, 3),
    input_tensor=None,
    include_top=False,
    weights="imagenet",
    pooling='avg',
)

350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


In [ ]:
def build_tl_model(
    input_shape=input_shape,
    output_shape=output_shape,
    learning_rate=learning_rate,
    dropout_rate=dropout_rate,
    seed=seed
):

  tf.random.set_seed(seed)

  # Define input layer with shape matching the input images
  inputs = tfk.Input(shape=input_shape, name='input_layer')

  # Augmentation
  augmentation = tf.keras.Sequential([
      tfkl.RandomFlip("horizontal_and_vertical"),
      tfkl.RandomBrightness(0.2),
      tfkl.RandomRotation((-1,1)),
      tfkl.RandomTranslation((-0.2, 0.2),(-0.2,0.2)),
      tfkl.RandomContrast(0.2)], name='preprocessing')

  x = augmentation(inputs)

  x = convbase(x)

  # FC (Fully Connected) - only trainable part
  x = tfkl.Dense(units=512, name='dense1')(x)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.Activation('relu', name='act1')(x)
  x = tfkl.Dropout(dropout_rate, seed=seed, name='dropout1')(x)

  x = tfkl.Dense(units=128, name='dense2')(x)
  x = tfkl.Activation('relu', name='act2')(x)
  x = tfkl.Dropout(dropout_rate, seed=seed, name='dropout2')(x)

  x = tfkl.Dense(units=output_shape, name='dense3')(x)
  outputs = tfkl.Activation('softmax', name='softmax')(x)

  # Create a Model connecting input and output
  model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  return model

In [ ]:
# Build the model with specified input and output shapes
tl_model = build_tl_model()

## 🧠 Train and Save the Model

In [ ]:
# freeze weights
for layer in convbase.layers:
  layer.trainable = False

# Compile the model with Categorical Cross-Entropy loss and Lion optimizer
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Lion(learning_rate), metrics=['accuracy'])

# Train model
tl_history = tl_model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=(24-5) #subtract the patience
).history

tl_model.save('weights.keras')

Epoch 1/24


I0000 00:00:1732384998.621234      68 service.cc:145] XLA service 0x7dc3cef27fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732384998.621286      68 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1732384998.910422      68 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


406/406 ━━━━━━━━━━━━━━━━━━━━ 175s 373ms/step - accuracy: 0.5158 - loss: 1.6062
Epoch 2/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8343 - loss: 0.5707
Epoch 3/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8576 - loss: 0.4948
Epoch 4/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8705 - loss: 0.4581
Epoch 5/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8821 - loss: 0.4279
Epoch 6/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8837 - loss: 0.4190
Epoch 7/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8917 - loss: 0.3986
Epoch 8/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8948 - loss: 0.3889
Epoch 9/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.8953 - loss: 0.3843
Epoch 10/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 369ms/step - accuracy: 0.9025 - loss: 0.3738
Epoch 11/24
406/406 ━━━━━━━━━━━━━━━━━━━━ 150s 370ms/step - accuracy: 0.8996 - loss: 0.3704
Epoch 12/24
406/406

## 🧠 Fine Tuning

In [ ]:
tl_model.load_weights('weights.keras')

#unfreeze weights
for layer in convbase.layers:
    layer.trainable = True

# Compile the model with Categorical Cross-Entropy loss and Lion optimizer
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Lion(1e-6), metrics=['accuracy'])

# Fine-tune the model
tl_history = tl_model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=(22-5) #subtract the patience
).history

tl_model.save('weights.keras')

# Delete the model to free up resources
del tl_model

Epoch 1/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 728s 2s/step - accuracy: 0.9309 - loss: 0.2862
Epoch 2/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9524 - loss: 0.2227
Epoch 3/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9598 - loss: 0.1984
Epoch 4/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9642 - loss: 0.1839
Epoch 5/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9670 - loss: 0.1722
Epoch 6/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9701 - loss: 0.1618
Epoch 7/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9739 - loss: 0.1549
Epoch 8/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9746 - loss: 0.1494
Epoch 9/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9773 - loss: 0.1412
Epoch 10/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9765 - loss: 0.1402
Epoch 11/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9779 - loss: 0.1363
Epoch 12/22
406/406 ━━━━━━━━━━━━━━━━━━━━ 

## 📊 Prepare Your Submission

To prepare your submission, create a `.zip` file that includes all the necessary code to run your model. It **must** include a `model.py` file with the following class:

```python
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""
```

The next cell shows an example implementation of the `model.py` file, which includes loading model weights from the `weights.keras` file and conducting predictions on provided input data. The `.zip` file is created and downloaded in the last notebook cell.

❗ Feel free to modify the method implementations to better fit your specific requirements, but please ensure that the class name and method interfaces remain unchanged.

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl


class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('weights.keras')

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

Writing model.py


In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py weights.keras

%cd /kaggle/working
from IPython.display import FileLink
FileLink(filename)

  adding: model.py (deflated 56%)
  adding: weights.keras (deflated 7%)
/kaggle/working


/kaggle/working/submission_241123_231414.zip